In [117]:
!pip install sense2vec

In [1]:
%reload_ext autoreload
%autoreload 2

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import copy
from pprint import pprint
from pathlib import Path
from typing import Any, Dict, List, Set, Tuple
import spacy
import srsly
# import recon
from recon.corpus import Corpus
from recon.constants import NONE
from recon.corrections import fix_annotations
from recon.dataset import Dataset
from recon.loaders import read_jsonl
from recon.types import Correction, Example, PredictionError, HardestExample, NERStats, EntityCoverageStats, EntityCoverage, Transformation, TransformationType, OperationState
from recon.stats import (
    get_ner_stats, get_entity_coverage, get_sorted_type_counts, get_probs_from_counts, entropy,
    calculate_entity_coverage_entropy, calculate_label_balance_entropy, calculate_label_distribution_similarity,
    detect_outliers
)
import recon.tokenization as tokenization
from recon.insights import get_ents_by_label, get_label_disparities, top_prediction_errors, top_label_disparities, get_hardest_examples
from recon.recognizer import SpacyEntityRecognizer
from recon.operations import registry
from recon.store import ExampleStore

In [3]:
# TODO: Fix Dataset loading with different file names
# train = Dataset("train").from_disk("./data/fashion_brands/fashion_brands_training.jsonl")
# dev = Dataset("dev").from_disk("./data/fashion_brands/fashion_brands_eval.jsonl")

corpus = Corpus.from_disk("./data/fashion_brands/", "fashion_brands")

In [4]:
print(corpus.apply(get_ner_stats, serialize=True).train)
print(corpus.apply(get_ner_stats, serialize=True).dev)
print(corpus.apply(get_ner_stats, serialize=True).all)

{
    "n_examples":1235,
    "n_examples_no_entities":930,
    "n_annotations":527,
    "n_annotations_per_type":{
        "FASHION_BRAND":527
    },
    "examples_with_type":null
}
{
    "n_examples":500,
    "n_examples_no_entities":371,
    "n_annotations":238,
    "n_annotations_per_type":{
        "FASHION_BRAND":238
    },
    "examples_with_type":null
}
{
    "n_examples":1735,
    "n_examples_no_entities":1301,
    "n_annotations":765,
    "n_annotations_per_type":{
        "FASHION_BRAND":765
    },
    "examples_with_type":null
}


In [5]:
ec = corpus.apply(get_entity_coverage, case_sensitive=True)
ec.train[:5]

[EntityCoverage(text='Nike', label='FASHION_BRAND', count=11, examples=[]),
 EntityCoverage(text='Uniqlo', label='FASHION_BRAND', count=11, examples=[]),
 EntityCoverage(text='Madewell', label='FASHION_BRAND', count=8, examples=[]),
 EntityCoverage(text='Bonobos', label='FASHION_BRAND', count=7, examples=[]),
 EntityCoverage(text='Gucci', label='FASHION_BRAND', count=7, examples=[])]

In [6]:
unique_ec = {e.text for e in ec.all}
len(unique_ec)

438

In [7]:
# source: https://www.apparelsearch.com/wholesale_clothing/popular_brand_names_clothes.htm
extra_brands = ["Adidas", "Aeffe S.P.A", "Agatha", "Agnes B", "", "Anna Osmushkina", "Anna Sui", "Aquascutum", "Armani Exchange", "Austin Reed", "Avirex", "BCBG", "Benetton", "Bisou-Bisou", "Body Glove", "Bogner", "Burton", "Brioni", "Calvin Klein", "Cesarani", "Champion", "Chanel", "Christian Dior", "", "Christian Lacoix", "Claiborne", "Club Monaco", "Columbia", "Converse", "Courrages", "Cutter & ", "Buck", "Diesel", "Dockers", "", "Dolce & Gabbana", "Donna Karan", "Ecco", "Ecko", "Eddie Bauer", "Ellesse", "Eliott & ", "Lucca", "Energie", "Esprit", "Everlast", "Fia Miami", "Fila", "Fiorelli", "", "Fratelli Corneliani", "Fred Perry", "Fruit of the ", "Loom", "Fubu", "", "Gianfranco Ferre", "Gianni Versace", "Giorgio Armani", "Gucci", "Guess", "Helly Hansen", "Hugo Boss", "J. Crew", "Izod", "Jitrois", "Jennifer Lopez", "", "Jenny Yoo", "Jhane Barnes", "Joe Boxer", "", "John Smedley", "Jordache", "Kenneth Cole ", "/ Reaction", "Lacoste", "Land's End", "", "La Perla", "Laura Ashley", "Lee", "Le Tigre", "Levi's", "Liz Claiborne", "L.L Bean", "", "Louis Feraud", "Lucky Brand ", "Jeans", "", "Madeleine Vionnet", "Mango", "Marc Jacobs", "", "Marcia Grachvogel", "", "Marianne Alvoni", "", "Michael Kors", "Moschino", "Mudd", "Munsingwear", "Nancy LordNew Balance", "Nicole Miller", "Nike", "", "Norma Kamali", "Oky-coky", "Oilily", "", "Olivier Strelli", "Oneill", "OP", "", "OshKosh B'Gosh", "Paul Fredrick", "Paul Shark", "Paul Smith", "", "Pelle Pelle", "Pepe Jeans", "Perry Ellis", "", "Perry Landhaus", "Pierre Cardin", "", "Pierre Garroudi", "Prada", "Puma", "Quiksilver", "Ralph Lauren", "Rampage", "Red Monkey", "Red or Dead", "Roberto Angelico", "Rocawear", "Russell", "Savane", "Salvatore J. ", "Cesarani", "Sean John", "Sinequanone", "Sisley", "Southpole", "Speedo", "Steven Alan", "Swatch", "Timberland", "Todd Oldham", "Tommy Hilfiger", "Van Heusen", "Vans", "Versace", "Vokal", "Wrangler", "Yves Saint ", "Laurent", "", "Z. Cavaricci", "Zanetti", "Zero"]
extra_brands = {eb for eb in extra_brands if eb != ""}
len(extra_brands)

141

In [8]:
unique_ec.update(extra_brands)

In [9]:
len(unique_ec)

556

In [10]:
from typing import Any, Callable, Dict, List, Optional

from recon.dataset import Dataset
from pydantic import root_validator
from recon.types import Example, Span, Token
import numpy as np
from recon.augmentation import substitute_spans
from recon.operations import operation, registry
from recon.preprocess import SpacyPreProcessor

import names
from snorkel.augmentation import transformation_function, ApplyAllPolicy
from snorkel.preprocess.nlp import SpacyPreprocessor
from recon.preprocess import SpacyPreProcessor
import spacy

In [11]:
nlp = spacy.load("en_core_web_sm")
spacy_pre = SpacyPreProcessor(nlp)

In [12]:
np.random.seed(0)

def augment_example(
    example: Example,
    span_f: Callable[[Span, Any], Optional[str]],
    spans: List[Span] = None,
    span_label: str = None,
    **kwargs: Any,
) -> List[Example]:

    if spans is None:
        spans = example.spans

    prev_example_hash = hash(example)
    example_t = None

    if span_label:
        spans = [s for s in spans if s.label == span_label]

    if spans:
        spans_to_sub = [np.random.choice(spans)]

        span_subs = {}
        for span in spans_to_sub:
            res = span_f(span, **kwargs)  #  type: ignore
            if res:
                span_subs[span] = res

        if any(span_subs.values()):
            res = substitute_spans(example, span_subs)
            if hash(res) != prev_example_hash:
                example_t = res

    return example_t

In [13]:
np.random.seed(0)


def ent_label_sub(
    example: Example, label: str, subs: List[str]
) -> Optional[Example]:
    
    def augmentation_f(span: Span, subs: List[str]) -> Optional[str]:
        subs = [s for s in subs if s != span.text]
        sub = None
        if len(subs) > 0:
            sub = np.random.choice(subs)
        return sub

    return augment_example(example, span_f=augmentation_f, span_label=label, subs=subs)


# replacement_names = [names.get_full_name() for _ in range(50)]


@transformation_function()
def brand_sub(example: Example):
    return ent_label_sub(example.copy(deep=True), label="FASHION_BRAND", subs=list(unique_ec))


# @transformation_function()
# def person_sub(example: Example):
#     return ent_label_sub(example.copy(deep=True), label="PERSON", subs=replacement_names)

@transformation_function()
def gpe_sub(example: Example):
    return ent_label_sub(example.copy(deep=True), label="GPE", subs=["Russia", "USA", "China"])

In [14]:
def kb_sub(
    example: Example, spans_to_aliases_map: Dict[Span, str]
) -> Optional[Example]:
    
    def augmentation_f(span: Span, spans_to_aliases_map: Dict[Span, List[str]]) -> Optional[str]:
        sub = None
        if span in spans_to_aliases_map:
            aliases = spans_to_aliases_map[span]

            if len(aliases) > 0:
                rand_alias = np.random.choice(aliases)
                index = aliases.index(rand_alias)
                del spans_to_aliases_map[span][index]
                sub = rand_alias

        return sub

    return augment_example(example, span_f=augmentation_f, span_label=label, subs=subs)


# @transformation_function()
# def skills_sub(example: Example):
    

In [15]:
import spacy
from sense2vec import Sense2VecComponent

s2v_path = "../../sense2vec/s2v_old/"
nlp = spacy.load("en_core_web_sm")
s2v = Sense2VecComponent(nlp.vocab).from_disk(s2v_path)
nlp.add_pipe(s2v)

In [16]:
doc = nlp(corpus.train[0].text)

In [18]:
for ph in doc._.s2v_phrases:
    try:
        most_similar = ph._.s2v_most_similar(3)
    except:
        most_similar = None
    print(ph)
    print(most_similar)

It
None
preference
[(('personal preference', 'NOUN'), 0.863), (('personal preferences', 'NOUN'), 0.8488), (('preferance', 'NOUN'), 0.8077)]
I
None
more natural the hair
None
the better the style
None
me
None
matte finish
[(('satin finish', 'NOUN'), 0.8949), (('glossy', 'ADJ'), 0.8901), (('matte look', 'NOUN'), 0.8879)]
hair
[(('beard', 'NOUN'), 0.8841), (('curly hair', 'NOUN'), 0.8841), (('stubble', 'NOUN'), 0.8836)]
it
None
place
None


In [28]:
ms = [(('beard', 'NOUN'), 0.8841), (('curly hair', 'NOUN'), 0.8841), (('stubble', 'NOUN'), 0.8836)]
ms[0][0][0]


'beard'

In [19]:
import numpy as np

In [161]:
def test_model_most_similar_cache(s2v):
    query = "beekeepers|NOUN"
    s2v.cache = {"indices": np.empty((2,1)), "scores": np.empty((0,0))}
    assert s2v.cache
    assert query in s2v
    indices = s2v.cache["indices"]
    # Modify cache to test that the cache is used and values aren't computed
    query_row = s2v.vectors.find(key=s2v.ensure_int_key(query))
    scores = np.array(s2v.cache["scores"], copy=True)  # otherwise not writable
    honey_bees_row = s2v.vectors.find(key="honey_bees|NOUN")
    beekeepers_row = s2v.vectors.find(key="Beekeepers|NOUN")
    for i in range(indices.shape[0]):
        for j in range(indices.shape[1]):
            if indices[i, j] == honey_bees_row:
                scores[i, j] = 2.0
            elif indices[i, j] == beekeepers_row:
                scores[i, j] = 3.0
    s2v.cache["scores"] = scores
    
    print("CACHE")
    print(s2v.cache)
    ((key1, score1), (key2, score2)) = s2v.most_similar([query], n=2)
    assert key1 == "honey_bees|NOUN"
    assert score1 == 2.0
    assert key2 == "Beekeepers|NOUN"
    assert score2 == 3.0
    
s2v.s2v.cache = None
_s2v = test_model_most_similar_cache(s2v.s2v)
_s2v.cache

CACHE
{'indices': array([], shape=(0, 0), dtype=float64), 'scores': array([], shape=(0, 0), dtype=float64)}


AssertionError: 

In [162]:
s2v.s2v.vectors.find(key="honey_bees|NOUN")

171186

In [ ]:
assert s2v.s2v.cache

In [138]:
len(s2v.s2v)

1195261

In [139]:
1195261 * 1195261

1428648858121

In [17]:
from nltk.corpus import wordnet as wn

def get_synonym(word, pos=None):
    """Get synonym for word given its part-of-speech (pos)."""
    synsets = wn.synsets(word, pos=pos)
    # Return None if wordnet has no synsets (synonym sets) for this word and pos.
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        if words[0].lower() != word.lower():  # Skip if synonym is same as word.
            # Multi word synonyms in wordnet use '_' as a separator e.g. reckon_with. Replace it with space.
            return words[0].replace("_", " ")


# @operation("recon.v1.augment.replace_pos_with_synonym", pre=[spacy_pre])
def replace_pos_with_synonym(
    example: Example, 
    pos: str, 
    synonym_f: Callable[[str], str] = get_synonym
):

    pos_map = {
        "VERB": "v",
        "NOUN": "n",
        "ADJ": "a"
    }
    
    if pos not in pos_map:
        raise ValueError(f"Argument `pos` of {pos} not in {''.join(pos_map.keys())}")

    doc = example.doc
    span_starts = [s.start for s in example.spans]

    # Get indices of verb tokens in sentence.
    pos_idxs = [i for i, token in enumerate(doc) if token.pos_ == pos and token.idx not in span_starts]
    tokens = [doc[idx] for idx in pos_idxs]
    spans = [Span(text=token.text, start=token.idx, end=token.idx + len(token.text), label="") for token in tokens]

    def augmentation_f(span: Span, synonym_f: Callable[[str], str] = synonym_f) -> Optional[str]:
        return synonym_f(span.text)

    return augment_example(
        example,
        augmentation_f,
        spans=spans,
    )
    

spacy_pre = SpacyPreprocessor(text_field="text", doc_field="doc")    

@transformation_function(pre=[spacy_pre])
def replace_verb_with_synonym(example: Example):
    return replace_pos_with_synonym(example, "VERB")

@transformation_function(pre=[spacy_pre])
def replace_noun_with_synonym(example: Example):
    return replace_pos_with_synonym(example, "NOUN")

@transformation_function(pre=[spacy_pre])
def replace_adj_with_synonym(example: Example):
    return replace_pos_with_synonym(example, "ADJ")



In [23]:
s2v_pre = SpacyPreprocessor(text_field="text", doc_field="doc")
s2v_pre._nlp = nlp


@transformation_function(pre=[s2v_pre])
def replace_s2v(example):
    
    doc = example.doc
    span_starts = [s.start for s in example.spans]

    most_similar_map = {}
    for ph in doc._.s2v_phrases:
        try:
            most_similar = ph._.s2v_most_similar(5)
        except:
            most_similar = None
            
        if most_similar:
            span = Span(text=ph.text, start=ph.start_char, end=ph.end_char, label="")
            if span.start not in span_starts:
                most_similar_map[span] = most_similar
                
    
    def augmentation_f(span: Span, most_similar_map: Dict[Span, List[str]]) -> Optional[str]:
        choice = np.random.choice(most_similar_map[span][0][0])
        return choice

    return augment_example(
        example,
        augmentation_f,
        spans=list(most_similar_map.keys()),
        most_similar_map=most_similar_map
    )
    

In [24]:
tfs = [
    brand_sub,
    replace_s2v
#     person_sub,
#     gpe_sub
]

np.random.seed(0)

from snorkel.augmentation import ApplyOnePolicy, RandomPolicy

random_policy = RandomPolicy(
    len(tfs), sequence_length=2, n_per_original=2, keep_original=True
)

random_policy.generate_for_example()


policy = ApplyAllPolicy(len(tfs))
policy.generate_for_example()

[[], [0, 1]]

In [25]:
from tqdm import tqdm

from snorkel.augmentation.apply.core import BaseTFApplier


class ReconDatasetTFApplier(BaseTFApplier):
    
    def __init__(self, tfs, policy, span_label: str = None, sub_prob: float = 0.5):
        super().__init__(tfs, policy)
        self.span_label = span_label
        self.sub_prob = sub_prob
    
    def _apply_policy_to_data_point(self, x: Example) -> List[Example]:
        
        x_transformed = set()
        for seq in self._policy.generate_for_example():
            x_t = x.copy(deep=True)
            # Handle empty sequence for `keep_original`
            transform_applied = len(seq) == 0
            # Apply TFs
            for tf_idx in seq:
                tf = self._tfs[tf_idx]                
                x_t_or_none = tf(x_t)
                # Update if transformation was applied
                if x_t_or_none is not None:
                    transform_applied = True
                    x_t = x_t_or_none.copy(deep=True)
            # Add example if original or transformations applied
            if transform_applied:
                x_transformed.add(x_t)
        return list(x_transformed)


    def apply(self, ds: Dataset, progress_bar: bool = True) -> Dataset:
        
        @operation("recon.v1.augment")
        def augment(example: Example):
            transformed_examples = self._apply_policy_to_data_point(example)
            return transformed_examples
            
        ds.apply_("recon.v1.augment")
        
        return ds

In [26]:
len(corpus.train_ds)

1235

In [27]:
np.random.seed(0)

applier = ReconDatasetTFApplier(tfs, random_policy)
applier.apply(corpus.train_ds)

=> Applying operation 'recon.v1.augment' to dataset 'train'


KeyboardInterrupt: 

In [ ]:
len(corpus.train_ds)

In [191]:
corpus.train[:3]

[Example(text="It's all preference for which looks better, personally I feel that the more natural the hair looks the better the style, which for me means going with a matte finish which leaves the hair looking as natural as possible while still holding it in place", spans=[], tokens=[Token(text='It', start=0, end=2, id=0), Token(text="'s", start=2, end=4, id=1), Token(text='all', start=5, end=8, id=2), Token(text='preference', start=9, end=19, id=3), Token(text='for', start=20, end=23, id=4), Token(text='which', start=24, end=29, id=5), Token(text='looks', start=30, end=35, id=6), Token(text='better', start=36, end=42, id=7), Token(text=',', start=42, end=43, id=8), Token(text='personally', start=44, end=54, id=9), Token(text='I', start=55, end=56, id=10), Token(text='feel', start=57, end=61, id=11), Token(text='that', start=62, end=66, id=12), Token(text='the', start=67, end=70, id=13), Token(text='more', start=71, end=75, id=14), Token(text='natural', start=76, end=83, id=15), Token

In [97]:
print(corpus.apply(get_ner_stats, serialize=True).train)
print(corpus.apply(get_ner_stats, serialize=True).dev)
print(corpus.apply(get_ner_stats, serialize=True).all)

{
    "n_examples":2793,
    "n_examples_no_entities":1960,
    "n_annotations":1443,
    "n_annotations_per_type":{
        "FASHION_BRAND":1443
    },
    "examples_with_type":null
}
{
    "n_examples":500,
    "n_examples_no_entities":371,
    "n_annotations":238,
    "n_annotations_per_type":{
        "FASHION_BRAND":238
    },
    "examples_with_type":null
}
{
    "n_examples":3293,
    "n_examples_no_entities":2331,
    "n_annotations":1681,
    "n_annotations_per_type":{
        "FASHION_BRAND":1681
    },
    "examples_with_type":null
}


In [101]:
corpus.train[:5]

[Example(text="It's all preference for which looks better, personally I feel that the more natural the hair looks the better the style, which for me means going with a matte finish which leaves the hair looking as natural as possible while still retention it in place", spans=[], tokens=[Token(text='It', start=0, end=2, id=0), Token(text="'s", start=2, end=4, id=1), Token(text='all', start=5, end=8, id=2), Token(text='preference', start=9, end=19, id=3), Token(text='for', start=20, end=23, id=4), Token(text='which', start=24, end=29, id=5), Token(text='looks', start=30, end=35, id=6), Token(text='better', start=36, end=42, id=7), Token(text=',', start=42, end=43, id=8), Token(text='personally', start=44, end=54, id=9), Token(text='I', start=55, end=56, id=10), Token(text='feel', start=57, end=61, id=11), Token(text='that', start=62, end=66, id=12), Token(text='the', start=67, end=70, id=13), Token(text='more', start=71, end=75, id=14), Token(text='natural', start=76, end=83, id=15), Tok

In [114]:
# corpus.train_ds.to_disk("./fixed_data/fashion_brands_4_augmentations/train", force=True)

raw_data = [e.dict() for e in corpus.train_ds.data]
len(raw_data)
for e in raw_data:
    if 'doc' in e:
        del e['doc']

srsly.write_jsonl("./fixed_data/fashion_brands_4_augmentations/train.jsonl", raw_data)

In [33]:
corpus.example_store[corpus.train_ds.operations[0].transformations[2].example]

Example(text="Ooh, that was my shirt! It's Revolve.", spans=[Span(text='Revolve', start=29, end=36, label='FASHION_BRAND', token_start=9, token_end=9, kb_id=None)], tokens=[Token(text='Ooh', start=0, end=3, id=0), Token(text=',', start=3, end=4, id=1), Token(text='that', start=5, end=9, id=2), Token(text='was', start=10, end=13, id=3), Token(text='my', start=14, end=16, id=4), Token(text='shirt', start=17, end=22, id=5), Token(text='!', start=22, end=23, id=6), Token(text='It', start=24, end=26, id=7), Token(text="'s", start=26, end=28, id=8), Token(text='anachronorm', start=29, end=40, id=9), Token(text='.', start=40, end=41, id=10)], meta={'section': 'malefashionadvice'}, formatted=True, answer='accept', _view_id='ner_manual', _input_hash=-187997482, _task_hash=-284841919, _session_id=None)

In [34]:
len(corpus.train_ds)

1540

In [41]:
print(corpus.apply(get_ner_stats, serialize=True).train)
print(corpus.apply(get_ner_stats, serialize=True).dev)
print(corpus.apply(get_ner_stats, serialize=True).all)

{
    "n_examples":1540,
    "n_examples_no_entities":930,
    "n_annotations":1054,
    "n_annotations_per_type":{
        "FASHION_BRAND":1054
    },
    "examples_with_type":null
}
{
    "n_examples":500,
    "n_examples_no_entities":371,
    "n_annotations":238,
    "n_annotations_per_type":{
        "FASHION_BRAND":238
    },
    "examples_with_type":null
}
{
    "n_examples":2040,
    "n_examples_no_entities":1301,
    "n_annotations":1292,
    "n_annotations_per_type":{
        "FASHION_BRAND":1292
    },
    "examples_with_type":null
}


In [37]:
corpus.to_disk("./fixed_data/fashion_brands_ent_label_augment", force=True)

[autoreload of recon.operations failed: Traceback (most recent call last):
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/home/kabirkhan/Documents/recon/.venv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 276, in update_instances
    refs = gc.get_referrers(old)
KeyboardInterrupt
]
[autoreload of recon.types failed: Traceback (most recent call last):